# Discharge 1 Day all steps


First we are going to open a CSV File containing EFAS Stations.
We will select a random station.

In [2]:
import pandas as pd
from random import sample
stations = pd.read_csv('../static/EFAS_calib_stations_metadata.csv',encoding='iso-8859-1')
station = stations.sample(n=1)
station

,ObsID,StationName,Provider ID,Country code,StationLat,StationLon,Height,Height Units,DrainingArea.km2.Provider,Catchment Area Units,...,DiffDays_hist_6,DiffDays_nrt_6,cal_hist_24h,cal_nrt_24h,cal_hist_6h,cal_nrt_6h,CAL_TYPE,Notes,EC_calib,Dam/Lake
86,153,Lobith,1008,NL,51.844444,6.108333,13,m,160800,km2,...,364,3597,TRUE,TRUE,FALSE,TRUE,NRT_6h,-,3,-


We will open a dataset containing the ECMWF Ensemble Forecast (50 members) containg River Dicharge over 24 hours from Step 0 to 360 from the forecast 15th November 2018 00Z

In [3]:
import xarray as xr
ds = xr.open_dataset('../data/eue_15111800.nc')

We can quickly see the variable's of the file using ds.data_vars

In [4]:
ds.data_vars

Data variables:
    dis24                         (number, step, y, x) float32 ...
    lambert_azimuthal_equal_area  int32 ...
    land_binary_mask              (y, x) int8 ...
    upArea                        (y, x) float32 ...

We can check the data_variables dimensions using dis.dims

Number being the number of the Model

In [5]:
ds.dims

Frozen(SortedKeysDict({'y': 950, 'x': 1000, 'number': 50, 'step': 15}))

Now we know the variable and the dimensions we can look at the shape of the variable.

In [6]:
ds.dis24.shape

(50, 15, 950, 1000)

Now we will open the Historical Simulations

In [10]:
clim = xr.open_dataset('../data/clim_151118.nc')
clim


<xarray.Dataset>
Dimensions:                       (time: 15, x: 1000, y: 950)
Coordinates:
  * y                             (y) float64 5.498e+06 5.492e+06 ... 7.525e+05
  * x                             (x) float64 2.502e+06 2.508e+06 ... 7.498e+06
  * time                          (time) datetime64[ns] 2018-11-15T06:00:00 ... 2018-11-29T06:00:00
    step                          timedelta64[ns] ...
    surface                       int64 ...
    latitude                      (y, x) float32 ...
    longitude                     (y, x) float32 ...
    valid_time                    (time) datetime64[ns] ...
Data variables:
    dis24                         (time, y, x) float32 ...
    lambert_azimuthal_equal_area  int32 ...
    land_binary_mask              (y, x) int8 ...
    upArea                        (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    G

Now we will plot a time series of the derived station by using its Lat and Lon, locating the nearest point

In [8]:
inputEPSG = pyproj.Proj(init='epsg:4326')   # ref susyem EFAS-IS
outputEPSG = pyproj.Proj(init='epsg:3035')  # ref system LISFLOOD

x_lsf, y_lsf = pyproj.transform(inputEPSG, outputEPSG, float(station.StationLon.values), float(station.StationLat.values))

# extract data for selected point in netcdf file by LISFLOOD coordinates
dsloc = ds.sel(x=x_lsf,y=y_lsf,method='nearest')
climloc = clim.sel(x=x_lsf,y=y_lsf,method='nearest')

NameError: name 'pyproj' is not defined

Now that we have a location in the dsloc and climloc variables we can plot the data.
Below we will first align the data so that the time fields are in the same format and reference

Historical Simulations are based on the 06UTC of the Date in the Time field
Forecast fields are based on the timestep from the basetime.

In [47]:
base=dsloc.time.values
time=np.array([base+np.timedelta64(step) for step in dsloc.step.values])

We will now construct a dataframe consisting of a few Ensemble Members

Here we have chosen 3 Enemble Members

In [ ]:
df=pd.DataFrame({
    'TimeStep': time, 
    'Historical_times':climloc.time.values, 
    'Historical': climloc.dis24.values, 
    'Ensemble_Member_1': dsloc.dis24[0,:].values, 
    'Ensemble_Member_10': dsloc.dis24[10,:].values, 
    'Ensemble_Member_20': dsloc.dis24[20,:].values})


In [ ]:
Now we will plot the members o

In [1]:
%matplotlib notebook
import matplotlib
#matplotlib.use("TkAgg")
import matplotlib.pyplot as plot

matplotlib.rcParams['figure.figsize'] = [10, 10]

plot.figure(figsize=(10,10),num='EFAS Data in matplotlib with Jupyter')

plot.xlabel('TimeStep in 24h')
plot.ylabel(dsloc.dis24.GRIB_name+' ' + dsloc.dis24.GRIB_units)

plot.plot( 'Historical_times', 'Historical', data=df, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=2)
plot.plot( 'TimeStep', 'Ensemble_Member_1', data=df, marker='o', color='blue', linewidth=2, linestyle='dashed')
plot.plot( 'TimeStep', 'Ensemble_Member_10', data=df, marker='o', color='green', linewidth=2, linestyle='dashed')
plot.plot( 'TimeStep', 'Ensemble_Member_20', data=df, marker='o', color='olive', linewidth=2, linestyle='dashed')
plot.legend()
plot.show()

<IPython.core.display.Javascript object>

NameError: name 'dsloc' is not defined

In [11]:
clim.time

<xarray.Dataset>
Dimensions:                       (time: 15, x: 1000, y: 950)
Coordinates:
  * y                             (y) float64 5.498e+06 5.492e+06 ... 7.525e+05
  * x                             (x) float64 2.502e+06 2.508e+06 ... 7.498e+06
  * time                          (time) datetime64[ns] 2018-11-15T06:00:00 ... 2018-11-29T06:00:00
    step                          timedelta64[ns] ...
    surface                       int64 ...
    latitude                      (y, x) float32 ...
    longitude                     (y, x) float32 ...
    valid_time                    (time) datetime64[ns] ...
Data variables:
    dis24                         (time, y, x) float32 ...
    lambert_azimuthal_equal_area  int32 ...
    land_binary_mask              (y, x) int8 ...
    upArea                        (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    G

In [12]:
clim.step

<xarray.DataArray 'step' ()>
array(86400000000000, dtype='timedelta64[ns]')
Coordinates:
    step     timedelta64[ns] 1 days
    surface  int64 ...
Attributes:
    long_name:      time since forecast_reference_time
    standard_name:  forecast_period